<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_MixedData_11classes_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Check if the dataset zip file exists in Google Drive
import os
dataset_path = '/content/drive/MyDrive/SynDD2_mixed_640.zip'  # Adjust the path if needed
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"The dataset file at {dataset_path} was not found!")
print("Dataset zip file exists.")

Dataset zip file exists.


In [4]:
# Unzipping the dataset
import zipfile
extract_path = '/content/SynDD2_mixed_640'
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset unzipped successfully!")

Dataset unzipped successfully!


In [5]:
# Check the actual folder structure after unzipping
dataset_actual_path = os.path.join(extract_path, "Split")  # Adjust to the nested folder
if not os.path.exists(os.path.join(dataset_actual_path, "train")):
    raise FileNotFoundError("Train folder not found in the dataset path!")

In [6]:
# Install the YOLOv8 library
!pip install ultralytics --upgrade

# Import YOLO
from ultralytics import YOLO
import torch
import torch.nn as nn
import torch.nn.functional as F

print("Ultralytics library imported successfully!")

# Verify the dataset structure
from ultralytics.data.utils import check_cls_dataset
dataset_info = check_cls_dataset(dataset_actual_path)
print(f"Dataset verified: {dataset_info}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [7]:
# Count the number of training and validation images
import glob
train_images = glob.glob(os.path.join(dataset_actual_path, "train", "*", "*.jpg"))
val_images = glob.glob(os.path.join(dataset_actual_path, "val", "*", "*.jpg"))

if not train_images or not val_images:
    raise ValueError("Training or validation images were not found. Check dataset paths.")
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

Number of training images: 106991
Number of validation images: 13374


In [8]:
# Define dataset path
DATASET_PATH = dataset_actual_path  # Path to the unzipped dataset
OUTPUT_PATH = '/content/drive/MyDrive/yolov8_cls_results_640'

# ✅ **Define Focal Loss**
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        """
        gamma: Focusing parameter (higher = more focus on hard examples)
        alpha: Class balancing factor (set to None or a tensor of class weights)
        reduction: 'mean' or 'sum'
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Compute Focal Loss.
        inputs: raw logits (before softmax)
        targets: true labels (as class indices)
        """
        log_probs = F.log_softmax(inputs, dim=-1)  # Convert logits to log probabilities
        probs = torch.exp(log_probs)  # Convert log probabilities to actual probabilities

        # Get the probability corresponding to the target class
        target_probs = probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        # Compute Focal Loss
        focal_factor = (1 - target_probs) ** self.gamma
        loss = -focal_factor * log_probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        if self.alpha is not None:
            alpha_factor = self.alpha.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)
            loss *= alpha_factor

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# ✅ **Define class imbalance handling using alpha (optional)**
alpha = torch.tensor([
    1.058,  # Drinking
    4.605,  # Eating
    1.058,  # Normal
    1.058,  # Phone_Call_(Left)
    1.058,  # Phone_Call_(Right)
    1.058,  # Reaching_Behind
    1.058,  # Sleep
    1.000,  # Smoking (reference class)
    1.058,  # Text_(Left)
    1.058,  # Text_(Right)
    1.058   # Yawning
], dtype=torch.float32).to("cuda")  # Move to GPU if available

# ✅ **Initialize the YOLOv8 model**
model = YOLO('yolov8m-cls.pt')  # Use yolov8m-cls as a starting point
print("YOLOv8 model initialized with pretrained weights.")

# ✅ **Attach Focal Loss to YOLO's loss function**
model.loss = FocalLoss(gamma=2.0, alpha=alpha, reduction='mean')

# ✅ **Train the model**
model.train(
    data=DATASET_PATH,
    epochs=50,
    batch=32,
    imgsz=640,
    project=OUTPUT_PATH,
    name='yolov8_cls_640_focal',
    pretrained=True,
    optimizer='Adam',
    lr0=0.0005,  # **Lower learning rate to prevent bias**
)
print("Training completed!")

# ✅ **Evaluate the model**
results = model.val()
print("Validation completed!")

# ✅ **Save the best weights**
best_weights_path = f"{OUTPUT_PATH}/yolov8_cls_640_focal/weights/best.pt"
os.makedirs(f"{OUTPUT_PATH}/yolov8_cls_640_focal/weights", exist_ok=True)
!cp /content/runs/classify/yolov8_cls_640_focal/weights/best.pt {best_weights_path}
print(f"Best weights saved to {best_weights_path}")

100%|██████████| 32.7M/32.7M [00:00<00:00, 150MB/s]


YOLOv8 model initialized with pretrained weights.
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/content/SynDD2_mixed_640/Split, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8_cls_results_640, name=yolov8_cls_640_focal3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

100%|██████████| 5.35M/5.35M [00:00<00:00, 61.3MB/s]


AMP: checks passed ✅


train: Scanning /content/SynDD2_mixed_640/Split/train... 106991 images, 0 corrupt: 100%|██████████| 106991/106991 [00:31<00:00, 3400.41it/s]


...

      47/50      10.1G     0.0805         15        640: 100%|██████████| 3344/3344 [11:39<00:00,  4.78it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 209/209 [00:41<00:00,  5.07it/s]

                   all      0.996          1



      Epoch    GPU_mem       loss  Instances       Size


      48/50      10.1G    0.07868         32        640:  10%|▉         | 334/3344 [01:10<10:38,  4.71it/s]


KeyboardInterrupt: 